In [ ]:
import sys
import sys
import os
# makes it so that tensorflow doesn't print warnings on startup
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all logs, 1 = filter INFO, 2 = filter WARNING, 3 = only errors
import tensorflow as tf

In [ ]:
# If having trouble with pyenv and python versions, ensure that this print statement says 3.11.9
print("Python version: " + sys.version)
print("Tensorflow version: " + tf.__version__)